In [ ]:
# whisperインストール
!pip install git+https://github.com/openai/whisper.git 
!pip install openai 

In [ ]:
import subprocess
import openai

# # 音声データ抽出
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-acodec', 'libmp3lame', '-ab', '256k', 'audio.mp3'])

# # Whisper実行
subprocess.run(['whisper', 'audio.mp3', '--model', 'medium'])

with open('audio.srt', 'r', encoding='utf-8') as file:
    srt_data = file.read()

# gpt-3.5-turboで翻訳
openai.api_key = 'your_api_key'
completions = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": f"[以下の英文をsrt形式を維持して翻訳してください]:\n{srt_data}"}
    ],
)

srt_text = completions["choices"][0]["message"]["content"]

#　改行関数
def add_line_breaks_srt(srt_text):
    lines = srt_text.split("\n")
    result = ""

    for i, line in enumerate(lines):
        if line.strip().isdigit():
            result += line 
        elif "-->" in line:
            result += line 
        else:
            result += add_line_breaks(line) + "\n"

        # 空行を追加
        if i < len(lines) - 1 and lines[i + 1] != "" and line != "":
            result += "\n"

    return result.strip()

def add_line_breaks(text):
    result = ""
    line_length = 0

    for char in text:
        if char == "、" and line_length >= 7:
            result += "\n"
            line_length = 0
        elif char == "。":
            result += "\n"
            line_length = 0
        else:
            result += char
            line_length += 1

    return result

# 改行されたテキストを取得
result = add_line_breaks_srt(srt_text)
# ファイルに書き込む
with open("output.srt", "w", encoding='utf-8') as file:
    file.write(result)

print(result)
print("ファイルに書き込みました。")


# 字幕動画生成
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-vf', "subtitles=output.srt:force_style='FontName=Helvetica,FontSize=6'", 'test_subtitle.mp4'])

In [33]:
# 字幕動画生成
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-vf', "subtitles=output.srt:force_style='FontName=Helvetica,FontSize=6'", 'test_subtitle_bard.mp4'])

CompletedProcess(args=['ffmpeg', '-i', 'test.mp4', '-vf', "subtitles=output.srt:force_style='FontName=Helvetica,FontSize=6'", 'test_subtitle_bard.mp4'], returncode=0)